In [1]:
#Author: Kristen
#Description: For now this is just random exploration (first program)
#Last Update Date: July 23rd
#Dependencies: None

In [2]:
set.seed(42)

In [3]:
#Install the packages if you have not already installed them on your computer
#install.packages("RandPro")
#install.packages("corrplot")
#install.packages("changepoint")
#install.packages("ecp")

In [4]:
#Libraries
library(corrplot)
library(caret)
library(RandPro)
library(dplyr)
library(changepoint)
library(ecp)
library(ggplot2)

corrplot 0.84 loaded
Loading required package: lattice
Loading required package: ggplot2

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Successfully loaded changepoint package version 2.2.2
 NOTE: Predefined penalty values changed in version 2.2.  Previous penalty values with a postfix 1 i.e. SIC1 are now without i.e. SIC and previous penalties without a postfix i.e. SIC are now with a postfix 0 i.e. SIC0. See NEWS and help files for further details.
Loading required package: Rcpp


In [5]:
test <- read.csv("test.csv")
head(test, n= 2)

rn,tBodyAcc.mean.X,tBodyAcc.mean.Y,tBodyAcc.mean.Z,tBodyAcc.std.X,tBodyAcc.std.Y,tBodyAcc.std.Z,tBodyAcc.mad.X,tBodyAcc.mad.Y,tBodyAcc.mad.Z,⋯,fBodyBodyGyroJerkMag.meanFreq,fBodyBodyGyroJerkMag.skewness,fBodyBodyGyroJerkMag.kurtosis,angle.tBodyAccMean.gravity,angle.tBodyAccJerkMean.gravityMean,angle.tBodyGyroMean.gravityMean,angle.tBodyGyroJerkMean.gravityMean,angle.X.gravityMean,angle.Y.gravityMean,angle.Z.gravityMean
3,0.280,-0.0195,-0.113,-0.995,-0.967,-0.979,-0.997,-0.964,-0.977,⋯,0.4150,-0.391,-0.760,-0.119,0.178,0.101,0.809,-0.849,0.181,-0.0491
5,0.277,-0.0166,-0.115,-0.998,-0.981,-0.990,-0.998,-0.980,-0.990,⋯,0.0878,-0.351,-0.699,0.123,0.123,0.694,-0.616,-0.848,0.185,-0.0439


In [6]:
train <- read.csv("train.csv")
head(train, n =2)

rn,activity,tBodyAcc.mean.X,tBodyAcc.mean.Y,tBodyAcc.mean.Z,tBodyAcc.std.X,tBodyAcc.std.Y,tBodyAcc.std.Z,tBodyAcc.mad.X,tBodyAcc.mad.Y,⋯,fBodyBodyGyroJerkMag.meanFreq,fBodyBodyGyroJerkMag.skewness,fBodyBodyGyroJerkMag.kurtosis,angle.tBodyAccMean.gravity,angle.tBodyAccJerkMean.gravityMean,angle.tBodyGyroMean.gravityMean,angle.tBodyGyroJerkMean.gravityMean,angle.X.gravityMean,angle.Y.gravityMean,angle.Z.gravityMean
7,STANDING,0.279,-0.0196,-0.110,-0.997,-0.967,-0.983,-0.997,-0.966,⋯,0.146,-0.217,-0.5640,-0.2130,-0.231,0.0146,-0.190,-0.852,0.182,-0.0430
11,STANDING,0.277,-0.0127,-0.103,-0.995,-0.973,-0.985,-0.996,-0.974,⋯,0.121,0.349,0.0577,0.0807,0.596,-0.4760,0.116,-0.852,0.188,-0.0347


In [7]:
# Check some summary plots/stats
corTrain <- cor(train[, (3:100)])
#corrplot(corTrain, method = 'color',tl.pos = "n")


In [8]:
#plot(train$activity, las = 3, col = 'darkred')

In [9]:
#This code block is from the clusterv source code fro random projections available at:
#http://homes.di.unimi.it/valenti/SW/clusterv-source/rp.R

# Random projections to a lower dimension subspace with the Achlioptas' projection matrix
# The projection is performed using a  projection matrix P s.t. Prob(P[i,j]=sqrt(3))=Prob(P[i,j]=-sqrt(3)=1/6;
# Prob(P[i,j]=0)=2/3
# Input:
# d : subspace dimension
# m : data matrix (rows are features and columns are examples)
# scaling : if TRUE (default) scaling is performed
# Output:
# data matrix (dimension d X ncol(m)) of the examples projected in a d-dimensional random subspace
Achlioptas.random.projection <- function(d=2, m, scaling=TRUE){
  d.original <- nrow(m);
  if (d >= d.original)
    stop("norm.random.projection: subspace dimension must be lower than space dimension", call.=FALSE);
  # Projection matrix
  P <- floor(runif(d*d.original,1,7)); # generate a vector 1 to 6 valued
  sqr3 <- sqrt(3);
  P[P==1] <- sqr3;
  P[P==6] <- -sqr3;
  P[P==2 | P==3 | P==4 | P==5] <- 0;
  P <- matrix(P, nrow=d);
  
  # random data projection
  if (scaling == TRUE)
    reduced.m <- sqrt(1/d) * (P%*%m)
  else 
    reduced.m <- P%*%m;;
  reduced.m 
}

# Prediction of the dimension of random projection we need to obtain a given distortion according to JL lemma
# Input:
# n : cardinality of the data
# epsilon : distortion (0 < epsilon <= 0.5)
# Output:
# minimum dimension
JL.predict.dim <- function(n, epsilon=0.5) {
  d <- 4 * (log(n) / epsilon^2);
  ceiling(d)
}

In [10]:
n = length(train$activity)-2
JL.predict.dim(n, epsilon = 0.5)

[1] 132

In [11]:
trainMatrix <- t(data.matrix(train[,-(1:2)]))
rpResults<- data.frame(t(Achlioptas.random.projection(d = 3, trainMatrix, scaling = TRUE)))
head(rpResults)

X1,X2,X3
-2.56284,-5.06506,-25.35097
-2.80182,-2.74477,-26.73867
-5.70720,-4.43522,-22.22490
1.40127,-4.84300,-25.59307
-2.59708,-4.35027,-26.46122
-6.63617,-3.55107,-22.84730


In [12]:
#devisiveResults = e.divisive(data.matrix(rpResults), min.size = 5, sig.lvl = 0.5)

In [13]:
#hard code results in to avoid rerunning above code
devisiveResults = c(        1,        9,        25,        43,        52,        59,        74,        83,        101,        109,        114,        132,        141,        169,        199,        234,        257,        268,        277,        285,        294,        301,        311,        322,        331,        347,        369,        384,        396,        409,        417,        450,        459,        466,        473,        501,        514,        523,        534,        549,        565,        573,        599,        605,        614,        623,        650,        661,        668,        681,        693,        706,        715,        722,        743,        757,        769,        779,        807,        820,        828,        836,        848,        872,        880,        896,        912,        926,        935,        945,        955,        978,        989,        996,        1006,        1029,        1048,        1064,        1073,        1107,        1117,        1127,        1136,        1154,        1166,        1199,        1233,        1266,        1284,        1301,        1314,        1327,        1331,        1342,        1353,        1374,        1384,        1390,        1399,        1425,        1436,        1459,        1491,        1510,        1532,        1543,        1566,        1586,        1627,        1647,        1684,        1694,        1708,        1713,        1725,        1745,        1767,        1774,        1795,        1805,        1810,        1847,        1874,        1888,        1923,        1952,        1962,        1968,        1976,        1989,        2014,        2025,        2036,        2058,        2085,        2099,        2107,        2112,        2121,        2154,        2168,        2178,        2187,        2219,        2239,        2264,        2297,        2315,        2332,        2342,        2358,        2383,        2401,        2404,        2414,        2436,        2449,        2469,        2510,        2541,        2565,        2569,        2583,        2597,        2607,        2628,        2654,        2666,        2673,        2681,        2696,        2712,2731,        2744,        2759,        2767,        2786,        2790,        2799,        2816,        2830,        2840,        2852,        2855,        2864,        2875,        2878,        2887,        2913,        2920,        2925,        2935,        2953,        2962,        2974,        2983,        2988,        2995,        3016,        3041,        3066,        3079,        3097,        3104,        3130,        3145,        3159,        3182,        3191,        3212,        3221,        3239,        3255,        3290,        3317,        3349,        3375,        3389,        3399,        3407,        3419,        3445,        3467,        3491,        3512,        3522,        3555,        3572,        3587,3610)


In [14]:
# Get actual changepoints
train$changepoint[1] = TRUE
for (x in 2:length(train$activity)){
train$changepoint[x] = (train$activity[x] != train$activity[x-1])
}
pointsChange <- which(train$changepoint, arr.ind = TRUE)


In [15]:
predictedCP = devisiveResults
actualCP = pointsChange
#plot(rpResults[1:100,'X1'],type="l", col=alpha("black", 0.8), lty = 3, ann=FALSE,ylim=range(-1:1)) 

#abline(v = actualCP, col = "blue")
#abline(v = predictedCP, col = "red", lty = 2)

In [16]:
#Create a matrix with the differences between each of the actual and predicted points
i = 1
j = 1
accuracyCP = matrix(nrow = length(predictedCP), ncol = length(actualCP))
for(i in 1:length(predictedCP)){ # iterate over each row
    for(j in 1:length(actualCP)){ # iterate over each column
        accuracyCP[i,j]=abs(predictedCP[i]-actualCP[j])
    }
}
head(accuracyCP)

#Create 2-norm function (sum of squares error function)
norm_2 <- function(x) sqrt(sum(x^2))

#Compute the sum of squared error
print("Sum or Squares Error:")
norm_2(apply(accuracyCP,2,min))
#Compute the mean squared error
print("Mean Squared Error:")
norm_2(apply(accuracyCP,2,min))/length(actualCP)

0,8,15,26,41,51,60,69,71,76,⋯,3521,3532,3542,3554,3560,3571,3586,3592,3598,3608
8,0,7,18,33,43,52,61,63,68,⋯,3513,3524,3534,3546,3552,3563,3578,3584,3590,3600
24,16,9,2,17,27,36,45,47,52,⋯,3497,3508,3518,3530,3536,3547,3562,3568,3574,3584
42,34,27,16,1,9,18,27,29,34,⋯,3479,3490,3500,3512,3518,3529,3544,3550,3556,3566
51,43,36,25,10,0,9,18,20,25,⋯,3470,3481,3491,3503,3509,3520,3535,3541,3547,3557
58,50,43,32,17,7,2,11,13,18,⋯,3463,3474,3484,3496,3502,3513,3528,3534,3540,3550


[1] "Sum or Squares Error:"


[1] 122.7721

[1] "Mean Squared Error:"


[1] 0.3076996

In [18]:
# Create a function to get the minimum of each column 
# which is the distance between an actual changepoint and its closest predicted changepoint
#which is the distance between an actual change point and the closest 
distData = data.frame(dist = apply(accuracyCP,2,min))

In [19]:
#ggplot(data=distData, aes(distData$dist)) + 
 # geom_histogram(aes(y =..density..), 
  #               bins = 15,
   ##              col="black", 
     #            fill="darkred", 
     #            alpha=.8) + 
  #geom_density(col="grey50", lty = 2) + 
  #labs(title="", x="Distance to Closest Predicted Change Point", y="Frequency")

In [20]:
#For each row, determine for each row what the activity number is (1 until the first changepoint, then 2, and so on)
currentActNum = 1
train$predActNum[1] = currentActNum
rpResults$predActNum[1] = currentActNum
for(i in 2:length(rpResults$X1)){
    train$predActNum[i] = currentActNum
    rpResults$predActNum[i] = currentActNum
    if(i %in% devisiveResults){
        currentActNum = currentActNum + 1
    }
}
#head(train, n = 40)

In [21]:
# The following code was modified from http://www.di.fc.ul.pt/~jpn/r/fourier/fourier.html
# returns the x.n time series for a given time sequence (ts) and
# a vector with the amount of frequencies k in the signal (X.k)
get.trajectory <- function(X.k,ts,acq.freq) {
  
  N   <- length(ts)
  i   <- complex(real = 0, imaginary = 1)
  x.n <- rep(0,N)           # create vector to keep the trajectory
  ks  <- 0:(length(X.k)-1)
  
  for(n in 0:(N-1)) {       # compute each time point x_n based on freqs X.k
    x.n[n+1] <- sum(X.k * cos(2*pi*ks*n/N)) / N # change this to the cos function
  }
  
  x.n * acq.freq 
}

In [22]:
# cs is the vector of complex points to convert
#Explore FFT Results
convert.fft <- function(cs, sample.rate=1) {
  cs <- cs / length(cs) # normalize

  distance.center <- function(c)signif( Mod(c),        4)
  angle           <- function(c)signif( 180*Arg(c)/pi, 3)
  
  df <- data.frame(cycle    = 0:(length(cs)-1),
                   freq     = 0:(length(cs)-1) * sample.rate / length(cs),
                   strength = sapply(cs, distance.center),
                   delay    = sapply(cs, angle))
  df
}

#View the fourier cycles for a couple of the unidentified activities
convert.fft(fft(rpResults[which(rpResults$predActNum == 1),"X1"]))
convert.fft(fft(rpResults[which(rpResults$predActNum == 2),"X1"]))
convert.fft(fft(rpResults[which(rpResults$predActNum == 4),"X1"]))

cycle,freq,strength,delay
0,0.0000000,3.2300,180.0
1,0.1111111,0.6014,-52.9
2,0.2222222,1.1550,84.5
3,0.3333333,0.6929,-27.7
4,0.4444444,0.7740,-167.0
5,0.5555556,0.7740,167.0
6,0.6666667,0.6929,27.7
7,0.7777778,1.1550,-84.5
8,0.8888889,0.6014,52.9


cycle,freq,strength,delay
0,0.0000,2.7830,180.00
1,0.0625,1.2020,-86.80
2,0.1250,0.5461,9.13
3,0.1875,0.5700,-38.90
4,0.2500,0.3788,169.00
5,0.3125,0.2185,-80.80
6,0.3750,0.4744,103.00
7,0.4375,0.5291,-92.30
8,0.5000,0.2038,0.00
9,0.5625,0.5291,92.30


cycle,freq,strength,delay
0,0.0000000,3.8730,180.0
1,0.1111111,1.1290,95.2
2,0.2222222,0.4791,142.0
3,0.3333333,0.3298,19.2
4,0.4444444,0.8569,-96.2
5,0.5555556,0.8569,96.2
6,0.6666667,0.3298,-19.2
7,0.7777778,0.4791,-142.0
8,0.8888889,1.1290,-95.2


In [23]:
X.k = fft(rpResults[which(rpResults$predActNum == 1),"X1"])
X.k = fft(X.k, inverse = TRUE)/length(X.k)
time     <- 2.56                            # measuring time interval (seconds) from data description
acq.freq <- 50                          # data acquisition frequency (Hz) from data description
ts  <- seq(0,time-1/acq.freq,1/acq.freq) # vector of sampling time-points (s) 

x.n <- get.trajectory(X.k,ts,acq.freq)   # create time wave


In [24]:
#Fourier Series
length(x.n)

[1] 128

In [25]:
#Function to plot a harmonic series
#From http://www.di.fc.ul.pt/~jpn/r/fourier/fourier.html
#plot.harmonic <- function(Xk, i, ts, acq.freq, color="red") {
#  Xk.h <- rep(0,length(Xk))
 # Xk.h[i+1] <- Xk[i+1] # i-th harmonic
 #harmonic.trajectory <- get.trajectory(Xk.h, ts, acq.freq=acq.freq)
 # points(ts, harmonic.trajectory, type="l", col=color)
#}

In [26]:
#Plot the inverse fast fourier transform with its first 3 harmonic oscillations
#This will remove the imaginary parts which is ok because the imaginary parts are all 0.

#plot(ts,x.n,type="l",ylim=c(-5,4),lwd=2, xlab = "Time", ylab = "Position")
#abline(v=0:time,h=-2:4,lty=3); abline(h=0)

#plot.harmonic(X.k,1,ts,acq.freq,"red")
#plot.harmonic(X.k,2,ts,acq.freq,"green")
#plot.harmonic(X.k,3,ts,acq.freq,"blue")

In [27]:
a = table(rpResults$predActNum)

In [28]:
#Find table position with max length
which(a==max(a))
#Find max length
max(a)

108 158 
108 158

[1] 41

In [29]:
# Plot Fourier transform for max length activity
X.k.max = fft(rpResults[which(rpResults$predActNum == 158),"X1"])
X.k.max = fft(X.k.max, inverse = TRUE)/length(X.k.max)
time     <- 2.56                            # measuring time interval (seconds) from data description
acq.freq <- 50                          # data acquisition frequency (Hz) from data description
ts  <- seq(0,time-1/acq.freq,1/acq.freq) # vector of sampling time-points (s) 

x.n.max <- get.trajectory(X.k.max,ts,acq.freq)   # create time wave

#plot(ts,x.n.max,type="l",ylim=c(-8,8),lwd=2, main= "", xlab = "Time", ylab = "Position")
#abline(v=0:time,h=-2:4,lty=3); abline(h=0)

#plot.harmonic(X.k.max,1,ts,acq.freq,"red")
#plot.harmonic(X.k.max,2,ts,acq.freq,"green")
#plot.harmonic(X.k.max,3,ts,acq.freq,"blue")
#length(x.n.max)

In [30]:
#Get the three waves for each activity
#Create Matrix with one activity segment per row, one series per column. 
#This should give an n*3*128 matrix where n is the number of distinct activities and a is the number of 
activityMatrix = array(rep(0, length(unique(rpResults$predActNum))* 3*128), dim = c( 128, 3, length(unique(rpResults$predActNum))))
#for(i in 1:length(devisiveResults)){
#    activityMatrix[nrow(activityMatrix) + 1,] = list(hello="hallo",goodbye="auf wiedersehen", volume=20.2)
#}

In [31]:
dim(activityMatrix)

[1] 128   3 229

In [32]:
time     <- 2.56                         # measuring time interval (seconds) from data description
acq.freq <- 50                           # data acquisition frequency (Hz) from data description
ts  <- seq(0,time-1/acq.freq,1/acq.freq) # vector of sampling time-points (s) 

randPros = c("X1", "X2", "X3")
for(i in 1:3){
    for(j in 1:229){
        X.k.j = fft(rpResults[which(rpResults$predActNum == j),randPros[i]])
        X.k.j = fft(X.k.j, inverse = TRUE)/length(X.k.j)
        x.n.j <- get.trajectory(X.k.j,ts,acq.freq)   # create time wave
        activityMatrix[,i,j] = x.n.j
    }
}

In [33]:
i = 1
j = 1
time     <- 2.56                         # measuring time interval (seconds) from data description
acq.freq <- 50                           # data acquisition frequency (Hz) from data description
ts  <- seq(0,time-1/acq.freq,1/acq.freq) # vector of sampling time-points (s) 
randPros = c("X1", "X2", "X3")

In [34]:
X.k.j = fft(rpResults[which(rpResults$predActNum == j),randPros[i]])
X.k.j = fft(X.k.j, inverse = TRUE)/length(X.k.j)
x.n.j <- get.trajectory(X.k.j,ts,acq.freq)
x.n.j

[1] -11.3562182-0i -11.0439403-0i -10.1353158-0i  -8.7120760-0i  -6.9011085-0i
  [6]  -4.8615520-0i  -2.7687806-0i  -0.7970342-0i   0.8974242-0i   2.1908386-0i
 [11]   3.0026879-0i   3.3015854-0i   3.1058907+0i   2.4791796+0i   1.5212852+0i
 [16]   0.3560894+0i  -0.8824447+0i  -2.0643839+0i  -3.0765308+0i  -3.8324232-0i
 [21]  -4.2787071-0i  -4.3974407-0i  -4.2043791-0i  -3.7437562-0i  -3.0804484-0i
 [26]  -2.2906486+0i  -1.4522662+0i  -0.6362062+0i   0.1005230+0i   0.7192234+0i
 [31]   1.2011557+0i   1.5465024+0i   1.7710547+0i   1.9012855+0i   1.9686168+0i
 [36]   2.0037210+0i   2.0316180+0i   2.0681590+0i   2.1182510-0i   2.1759205-0i
 [41]   2.2260548-0i   2.2474540-0i   2.2166756-0i   2.1120889+0i   1.9175674+0i
 [46]   1.6253381+0i   1.2376452+0i   0.7670618+0i   0.2354682+0i  -0.3281238+0i
 [51]  -0.8905849+0i  -1.4181283-0i  -1.8797934-0i  -2.2505539-0i  -2.5137233-0i
 [56]  -2.6624252-0i  -2.7000149-0i  -2.6394551-0i  -2.5017539-0i  -2.3136596-0i
 [61]  -2.1048651-0i  -1.9050055-0i  -1.7407329+0i  -1.6331349+0i  -1.5957193+0i
 [66]  -1.6331349+0i  -1.7407329+0i  -1.9050055-0i  -2.1048651-0i  -2.3136596-0i
 [71]  -2.5017539-0i  -2.6394551-0i  -2.7000149-0i  -2.6624252-0i  -2.5137233-0i
 [76]  -2.2505539-0i  -1.8797934-0i  -1.4181283-0i  -0.8905849+0i  -0.3281238+0i
 [81]   0.2354682+0i   0.7670618+0i   1.2376452+0i   1.6253381+0i   1.9175674+0i
 [86]   2.1120889+0i   2.2166756-0i   2.2474540-0i   2.2260548-0i   2.1759205-0i
 [91]   2.1182510-0i   2.0681590+0i   2.0316180+0i   2.0037210+0i   1.9686168+0i
 [96]   1.9012855+0i   1.7710547+0i   1.5465024+0i   1.2011557+0i   0.7192234+0i
[101]   0.1005230+0i  -0.6362062+0i  -1.4522662+0i  -2.2906486+0i  -3.0804484-0i
[106]  -3.7437562-0i  -4.2043791-0i  -4.3974407-0i  -4.2787071-0i  -3.8324232-0i
[111]  -3.0765308+0i  -2.0643839+0i  -0.8824447+0i   0.3560894+0i   1.5212852+0i
[116]   2.4791796+0i   3.1058907+0i   3.3015854-0i   3.0026879-0i   2.1908386-0i
[121]   0.8974242-0i  -0.7970342-0i  -2.7687806-0i  -4.8615520-0i  -6.9011085-0i
[126]  -8.7120760-0i -10.1353158-0i -11.0439403-0i

In [35]:
#activityMatrix

In [36]:
length(unique(rpResults$predActNum))

[1] 229